In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
import os
from os.path import join
import pandas as pd
import pickle as pkl
import openai_multi_tool_use_parallel_patch
from auto_analytics.tool_chat_loop import tool_chat_loop,tool_chat_loop_2, shell, tool_chat_loop_2nb, tool_chat_loop_2nb_with_vision
from auto_analytics.supervisor_loop import supervisor_chat_loop_2nb, system_message_library
from auto_analytics.utils.format_utils import message_thread_render
from auto_analytics.utils.nbformat_utils import save_cells_to_nb
from nbformat.v4 import new_notebook, new_code_cell, new_output, new_markdown_cell
from auto_analytics.utils.nbformat_utils import create_code_cell_from_captured, save_cells_to_nb
from auto_analytics.utils.nbformat_utils import convert_notebook_to_html, convert_notebook_to_pdf

report_dir = r"/Users/binxuwang/Github/GPT-Auto-Data-Analytics/reports"

In [6]:
result_dir = join(report_dir, "housing_price_prediction")
os.makedirs(result_dir, exist_ok=True)

In [4]:
nbcells = []
messages = []

In [9]:
csvpath = "/Users/binxuwang/Github/GPT-Auto-Data-Analytics/table_data/Housing_Price_Data.csv"

In [10]:
table_descriptions = """
Description:
This dataset contains various features of residential properties along with their corresponding prices. It is suitable for exploring and analyzing factors influencing housing prices and for building predictive models to estimate the price of a property based on its attributes.
Usage:
This dataset can be used for exploratory data analysis to understand the relationships between different housing features and prices.
It can also be used to build machine learning models for predicting housing prices based on the given features.
"""
column_descriptions = """
Features:
price: The price of the property.
area: The total area of the property in square feet.
bedrooms: The number of bedrooms in the property.
bathrooms: The number of bathrooms in the property.
stories: The number of stories (floors) in the property.
mainroad: Indicates whether the property is located on a main road (binary: yes/no).
guestroom: Indicates whether the property has a guest room (binary: yes/no).
basement: Indicates whether the property has a basement (binary: yes/no).
hotwaterheating: Indicates whether the property has hot water heating (binary: yes/no).
airconditioning: Indicates whether the property has air conditioning (binary: yes/no).
parking: The number of parking spaces available with the property.
prefarea: Indicates whether the property is in a preferred area (binary: yes/no).
furnishingstatus: The furnishing status of the property (e.g., furnished, semi-furnished, unfurnished).
"""

### Supervisor chat loop

In [11]:
task_objective = """Perform explorative data analysis of this dataset, 
This the focus should be to understand the relationships between different housing features and prices.
and attribute the dependencies between the features and the price of the property.
The descriptive statistics of the dataset should be calculated and visualized.
First, pose potential questions that could be answered by analyzing this dataset
Organize these question by the type of analysis that would be required to answer them.
These questions will be send to data scientist to write code to answer. 
"""

messages, nbcells, cache = supervisor_chat_loop_2nb(f"Here is a dataset, that can be loaded from {csvpath}."\
                            f"the dataset description {table_descriptions}. "\
                            f"and the column desctiption {column_descriptions}.\n"\
                            f"Our objective is to {task_objective}", 
                            model_name="gpt-4-turbo-preview", 
                            MAX_ROUND=1, nbcells=nbcells, )

content = messages[-1].content
with open(f"{result_dir}/analysis_questions.md", "w") as file:
    file.write(content)

Based on the description of the data available in the housing price dataset,
here is a structured approach to generating and categorizing research questions
for analysis.

### Research Questions Generation

#### A. Descriptive Analysis
1. What is the average price of properties in the dataset?
2. How does the average number of bedrooms, bathrooms, and stories vary across
the dataset?
3. What is the distribution of properties with and without features such as
mainroad access, guestroom, basement, hot water heating, air conditioning, and
prefarea?
4. How do properties distribute across different furnishing statuses (furnished,
semi-furnished, unfurnished)?

#### B. Correlational Analysis
5. Is there a correlation between the area of the property and its price?
6. How do the number of bedrooms, bathrooms, and parking spaces relate to the
price of the property?
7. Does having a property on the mainroad influence its price?
8. How does the presence of a guest room, basement, hot water heati

In [12]:
content = messages[-1].content
# Write content to markdown file
with open(f"{result_dir}/analysis_questions.md", "w") as file:
    file.write(content)
pkl.dump((messages, nbcells, ), 
         open(f"{result_dir}/supervisor_messages.pkl", "wb"))

### Data analysis

In [13]:
# read the objetive from the markdown file
with open(f"{result_dir}/analysis_questions.md", "r") as file:
    supervisor_objective = file.read()

cache_arr = []
messages = []
datasci_task_objective = """Perform explorative data analysis of this dataset, 
This the focus should be to understand the relationships between different housing features and prices.
and attribute the dependencies between the features and the price of the property.
The descriptive statistics of the dataset should be calculated and visualized.
A list of specific questions that could be answered by analyzing this dataset is provided below.
These questions are organized by the type of analysis that would be required to answer them.
Please write code to answer these questions, and organize the results in the end.
"""
#Let's answer them by code one by one and form a final report. 
messages, nbcells, cache = tool_chat_loop_2nb_with_vision(
                f"""Here is a dataset, that can be loaded from {csvpath}.
                the dataset description {table_descriptions}. 
                and the column desctiption {column_descriptions}.\n
                Our overall objective is to {datasci_task_objective}, \n
                the specific questions are listed as follows, 
                {supervisor_objective}""", enable_vision=True, vision_token_count=1024,
                MAX_ROUND=15, nbcells=nbcells, model_name='gpt-3.5-turbo-1106',
                chat_history=messages, 
                )
cache_arr = cache_arr + cache
pkl.dump((messages, nbcells, cache_arr), open(f"{result_dir}/datasci_messages.pkl", "wb"))

Python Code executed:
```python
import pandas as pd

# Load the dataset
file_path = '/Users/binxuwang/Github/GPT-Auto-Data-Analytics/table_data/Housing_Price_Data.csv'
housing_data = pd.read_csv(file_path)

# Display the first few rows of the dataset
housing_data.head()
```
Execution Succeed:


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


The dataset has been successfully loaded, and here are the first few rows of the
dataset:

|    |   price |   area |   bedrooms |   bathrooms |   stories | mainroad   |
guestroom   | basement   | hotwaterheating   | airconditioning   |   parking |
prefarea   | furnishingstatus   |
|---:|-------:|-------:|-----------:|-----------:|----------:|:-----------|:----
--------|:-----------|:------------------|:------------------|----------:|:-----
------|:-------------------|
|  0 |13300000 |    7420 |          4 |           2 |         3 | yes        |
no          | no         | no                | yes               |         2 |
yes        | furnished          |
|  1 |12250000 |    8960 |          4 |           4 |         4 | yes        |
no          | no         | no                | yes               |         3 |
no         | furnished          |
|  2 |12250000 |    9960 |          3 |           2 |         2 | yes        |
no          | yes        | no                | no               

4766729.247706422

Python Code executed:
```python
# Calculate the average number of bedrooms, bathrooms, and stories
average_bedrooms = housing_data['bedrooms'].mean()
average_bathrooms = housing_data['bathrooms'].mean()
average_stories = housing_data['stories'].mean()
average_bedrooms, average_bathrooms, average_stories
```
Execution Succeed:


...

Python Code executed:
```python
# Use the dataset to build a predictive model for property price based on its features
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Split the dataset into features and target variable
X = housing_data.drop('price', axis=1)
y = housing_data['price']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Fit a linear regression model to the training set
model = LinearRegression()
model.fit(X_train, y_train)

# Calculate the R-squared score of the model
r_squared = model.score(X_test, y_test)
r_squared
```
Execution Succeed:


0.6611214250980092

Python Code executed:
```python
# Identify the most significant predictors of property price using the trained model
coefficient_values = dict(zip(X.columns, model.coef_))
coefficient_values
```
Execution Succeed:


{'area': 253.09238233115641,
 'bedrooms': 122097.93326250254,
 'bathrooms': 996973.6887054548,
 'stories': 433489.8004736496,
 'mainroad': 441026.69405702484,
 'guestroom': 402848.69142251374,
 'basement': 324747.71476086485,
 'hotwaterheating': 1210834.2338196863,
 'airconditioning': 871377.8862767214,
 'parking': 216798.68662765212,
 'prefarea': 693144.41045331,
 'furnishingstatus_furnished': 199480.68443143598,
 'furnishingstatus_semi-furnished': 92820.80757937072,
 'furnishingstatus_unfurnished': -292301.49201080657}

Here are the results for the predictive and comparative analysis questions:

#### C. Predictive Analysis
10. The R-squared score of the linear regression model for predicting property
prices based on property features is approximately 0.66, indicating a moderate
level of predictability.

11. The most significant predictors of property price and their respective
coefficients in the model are as follows:

    - Area: 253.09
    - Bedrooms: 122,097.93
    - Bathrooms: 996,973.69
    - Stories: 433,489.80
    - Mainroad: 441,026.69
    - Guestroom: 402,848.69
    - Basement: 324,747.71
    - Hotwaterheating: 1,210,834.23
    - Airconditioning: 871,377.89
    - Parking: 216,798.69
    - Prefarea: 693,144.41
    - Furnishingstatus (Furnished): 199,480.68
    - Furnishingstatus (Semi-furnished): 92,820.81
    - Furnishingstatus (Unfurnished): -292,301.49

### Summary of Results:
- The average price of properties in the dataset is approximately 4,766,729.25.
- The average number of bedrooms, b

In [15]:
messages, nbcells, cache = tool_chat_loop_2nb_with_vision(
f"""Nice, we shall move on to build predictive model of price based on other features.
Please try multiple models and report their preditive accuracy, tell me which one is the best.""", 
                enable_vision=True, vision_token_count=1024,
                MAX_ROUND=15, nbcells=nbcells, model_name='gpt-3.5-turbo-1106',
                chat_history=messages, 
                )
cache_arr = cache_arr + cache
pkl.dump((messages, nbcells, cache_arr), open(f"{result_dir}/datasci_messages.pkl", "wb"))

Python Code executed:
```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score

# Instantiate the models
models = {
    'Random Forest': RandomForestRegressor(),
    'Support Vector Machine': SVR(),
    'Lasso Regression': Lasso()
}

# Split the dataset into features and target variable
X = housing_data.drop('price', axis=1)
y = housing_data['price']

# Evaluate the models using cross-validation
model_scores = {}
for model_name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    model_scores[model_name] = scores.mean()

model_scores
```
Execution Succeed:


{'Random Forest': -12.182656572728165,
 'Support Vector Machine': -16.61274079289823,
 'Lasso Regression': -9.911939317432127}

Here are the results of evaluating multiple predictive models using cross-
validation:

#### Predictive Model Evaluation Results:
- Random Forest: -12.18
- Support Vector Machine: -16.61
- Lasso Regression: -9.91

The scores presented are the mean cross-validation scores of the predictive
models. Among the models evaluated, the Lasso Regression model has the highest
mean cross-validation score, with a score of approximately -9.91.

Based on these results, the Lasso Regression model appears to be the best
performing model in this context.
[No tool use. Finishing conversation.]


In [16]:
messages, nbcells, cache = tool_chat_loop_2nb_with_vision(
f"""These are still very bad scores, can you try to improve the model by feature engineering, or normalization, or other methods?""", 
                enable_vision=True, vision_token_count=1024,
                MAX_ROUND=15, nbcells=nbcells, model_name='gpt-3.5-turbo-1106',
                chat_history=messages, 
                )
cache_arr = cache_arr + cache
pkl.dump((messages, nbcells, cache_arr), open(f"{result_dir}/datasci_messages.pkl", "wb"))

To improve the predictive model performance, we can explore a few methods such
as feature engineering and normalization. Here are some steps we can take to
enhance the model:

1. **Feature Engineering**: We can create new features or modify existing
features to better capture the relationships between the predictors and the
target variable. For example, we can add interaction features, category
encoding, or create new derived features.

2. **Normalization**: Normalizing or standardizing numerical features can be
beneficial, especially for models that are sensitive to the scale of the input
features.

3. **Regularization**: Regularization techniques such as L1 or L2 regularization
can be applied to penalize large coefficients and prevent overfitting.

4. **Model Tuning**: Hyperparameter tuning for each model can be performed to
optimize model performance.

I will start by applying some feature engineering and normalization techniques
to improve the model's predictive accuracy.
Python Co

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished,totalAmenities
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,True,False,False,3
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,True,False,False,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,False,True,False,3
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,True,False,False,4
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,True,False,False,4


Python Code executed:
```python
from sklearn.preprocessing import StandardScaler

# Apply StandardScaler to the numerical features
scaler = StandardScaler()
numerical_features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'totalAmenities']
housing_data[numerical_features] = scaler.fit_transform(housing_data[numerical_features])

# Display the modified dataset
housing_data.head()
```
Execution Succeed:


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_furnished,furnishingstatus_semi-furnished,furnishingstatus_unfurnished,totalAmenities
0,13300000,1.046726,1.403419,1.421812,1.378217,1,0,0,0,1,1.517692,1,True,False,False,0.851417
1,12250000,1.757010,1.403419,5.405809,2.532024,1,0,0,0,1,2.679409,0,True,False,False,0.013832
2,12250000,2.218232,0.047278,1.421812,0.224410,1,0,1,0,0,1.517692,1,False,True,False,0.851417
3,12215000,1.083624,1.403419,1.421812,0.224410,1,0,1,0,1,2.679409,1,True,False,False,1.689003
4,11410000,1.046726,1.403419,-0.570187,0.224410,1,1,1,0,1,1.517692,0,True,False,False,1.689003


Python Code executed:
```python
# Split the dataset into features and target variable
X = housing_data.drop('price', axis=1)
y = housing_data['price']

# Evaluate the models using cross-validation with the new features
model_scores = {}
for model_name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    model_scores[model_name] = scores.mean()

model_scores
```
Execution Succeed:


/Users/binxuwang/miniforge3/envs/torch/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.659e+11, tolerance: 4.724e+10
  model = cd_fast.enet_coordinate_descent(


{'Random Forest': -12.508234642305126,
 'Support Vector Machine': -16.612871206218653,
 'Lasso Regression': -9.911965792642281}

After applying feature engineering and normalization techniques, the performance
of the predictive models remains relatively unchanged:

#### Predictive Model Evaluation Results (After Feature Engineering and
Normalization):
- Random Forest: -12.51
- Support Vector Machine: -16.61
- Lasso Regression: -9.91

Although the scores are similar, it's important to note that these results can
vary based on the specific features and target variable used, as well as the
hyperparameters and tuning of the models. It may be beneficial to explore other
models or further fine-tune the existing models to potentially improve their
predictive accuracy.

If you're interested, we can explore additional models, hyperparameter tuning,
or other advanced techniques to further enhance the model's performance.
[No tool use. Finishing conversation.]


In [17]:
save_cells_to_nb(nbcells, f"{result_dir}/Housing_price_analysis.ipynb", 
                 save_html=True, save_pdf=True);

Notebook saved to /Users/binxuwang/Github/GPT-Auto-Data-Analytics/reports/housing_price_prediction/Housing_price_analysis.ipynb
HTML report saved to /Users/binxuwang/Github/GPT-Auto-Data-Analytics/reports/housing_price_prediction/Housing_price_analysis.html
PDF report saved to /Users/binxuwang/Github/GPT-Auto-Data-Analytics/reports/housing_price_prediction/Housing_price_analysis.pdf
